In [1]:
import pandas as pd
import numpy as np
from store_sales import encode_features, get_mae_score_cross_validation
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb

In [2]:
data = pd.read_csv('../data/prepared_data/prepared_data.csv')
data.head()

,date,store_number,item_family,items_on_promotion,item_sales,city,state,store_type,store_cluster,oil_price,...,is_popular_cluster,is_non_popular_cluster,is_special_non_working_day,is_national_holiday,is_state_pichincha,is_state_manabi_or_pastaza,is_city_quito_or_cayambe,is_city_manta_or_puyo,is_store_type_A,number_of_days_since_earthquake
0,2013-01-01,1,AUTOMOTIVE,0,0.0,Quito,Pichincha,D,13,93.14,...,0,0,0,1,1,0,1,0,0,-1201
1,2013-01-01,1,BABY CARE,0,0.0,Quito,Pichincha,D,13,93.14,...,0,0,0,1,1,0,1,0,0,-1201
2,2013-01-01,1,BEAUTY,0,0.0,Quito,Pichincha,D,13,93.14,...,0,0,0,1,1,0,1,0,0,-1201
3,2013-01-01,1,BEVERAGES,0,0.0,Quito,Pichincha,D,13,93.14,...,0,0,0,1,1,0,1,0,0,-1201
4,2013-01-01,1,BOOKS,0,0.0,Quito,Pichincha,D,13,93.14,...,0,0,0,1,1,0,1,0,0,-1201


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 28 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   date                             object 
 1   store_number                     int64  
 2   item_family                      object 
 3   items_on_promotion               int64  
 4   item_sales                       float64
 5   city                             object 
 6   state                            object 
 7   store_type                       object 
 8   store_cluster                    int64  
 9   oil_price                        float64
 10  day_type                         object 
 11  holiday_status                   object 
 12  holiday_location                 object 
 13  holiday_description              object 
 14  is_holiday_transferred           bool   
 15  mean_sales_prev_month            float64
 16  is_during_oil_prices_falling     int64  
 17  is_popul

#### We decided to make a prediction for the last 15 days of the dataset

In [4]:
number_of_days_to_predict = 15
min_test_date = pd.to_datetime(data['date'].unique()[-number_of_days_to_predict])
train_data = data[pd.to_datetime(data['date']) < min_test_date]
test_data = data[pd.to_datetime(data['date']) >= min_test_date]

In [13]:
tscv = TimeSeriesSplit(n_splits=5)

data = train_data[(train_data['store_number'] == 30) & (train_data['item_family'] == 'AUTOMOTIVE')]
X = data.drop(['item_sales'], axis=1)
y = data['item_sales']
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    print(len(X_train), len(X_test))

279 278
557 278
835 278
1113 278
1391 278


In [10]:
tscv = TimeSeriesSplit(n_splits=5)
xgboost = xgb.XGBRegressor(max_depth=2, n_jobs=-1, random_state=42)
#models = [xgboost]
#models_names = ['XGBoost']


#all_sales = pd.DataFrame()
mae_scores = []
#mape_scores = []

for store_num in train_data['store_number'].unique():
    for item_family in train_data['item_family'].unique():
        data = train_data[(train_data['store_number'] == store_num) & (train_data['item_family'] == item_family)]
        X = data.drop(['item_sales'], axis=1)
        y = data['item_sales']
        for train_index, test_index in tscv.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            X_train_encoded, X_test_encoded = encode_features(X_train.copy(), X_test.copy())
            mae_score = get_mae_score_cross_validation(X_train_encoded, X_test_encoded, y_train, y_test, xgboost)
            mae_scores.append(mae_score)

print('Average MAE for XGBoost:', np.array(mae_scores).mean())

Average MAE for XGBoost: 107.32414814814815


In [31]:
train_data['item_sales'].mean()

356.8107777940146